In [1]:
import zipfile as zp
import geopandas as gp
import pandas as pd
import numpy as np
import shutil
import os
from IPython.display import clear_output, display
import time
import concurrent.futures as cf

In [ ]:
# Code for crawling the shared drive
lst = []
for path, currentDirectory, files in os.walk(r"\\192.168.1.151\ncfc_data3"):
    for file in files:
        lst.append(os.path.join(path, file))

In [9]:
def zip_read(zip_path, df):
    archive = zp.ZipFile(zip_path)
    file_list = archive.filelist
    os.makedirs(r"E:\New folder2", exist_ok=True)
    for file in file_list:
#         df = pd.DataFrame({'File_Name':[], 'Columns':[], 'Ext':[], 'Size(in mb)':[]})
        file_zip_path = file.filename
        name_wt_ext = file.filename.split('/')[-1]
        name, ext, size = name_wt_ext.split('.')[0], name_wt_ext.split('.')[-1], file.file_size
        if ext == 'shp':
            if '/' not in file_zip_path:
                for file in archive.namelist():
                    if '/' not in file:
                        archive.extract(file, r"E:\New folder2")
            else:
                shpfile_folder_path = '/'.join(file_zip_path.split('/')[:-1])+'/'
                for file in archive.namelist():
                    if file.startswith(shpfile_folder_path):
                        archive.extract(file, r"E:\New folder2")
#             path_wo_file = '/'.join(file_zip_path.split('/')[:-1])
            temp_df = gp.read_file(r"E:\New folder2\{}".format(file_zip_path))
            columns = set(temp_df.columns)
        elif ext in {'xlsx', 'xls'}:
            archive.extract(file_zip_path, path=r"E:\New folder2")
            temp_df = pd.read_excel(r"E:\New folder2\{}".format(file_zip_path))
            columns = set(temp_df.columns)
        else:
            columns = np.nan
        df.loc[df.shape[0]] = [name, columns, ext, size/(1024*1024), zip_path+'\\'+file_zip_path]
    shutil.rmtree(r'E:\New folder2', ignore_errors=True)
    return df

In [2]:
def excel_read(excel_path, df):
    ext = excel_path.split('.')[-1]
    if ext == 'xls':
        temp_df = pd.read_excel(excel_path, engine='xlrd')
    else:
        temp_df = pd.read_excel(excel_path, engine='openpyxl')
    name = excel_path.split('\\')[-1].split('.')[0]
    columns = set(temp_df.columns)
    size = os.path.getsize(excel_path)
    df.loc[0] = [name, columns, ext, size/(1024*1024), excel_path]
    return df

In [11]:
def shape_read(shape_path, df):
    temp_df = gp.read_file(shape_path)
    name = shape_path.split('\\')[-1].split('.')[0]
    columns = set(temp_df.columns)
    ext = shape_path.split('\\')[-1].split('.')[-1]
    size = os.path.getsize(shape_path)
    df.loc[0] = [name, columns, ext, size/(1024*1024), shape_path]
    return df

In [12]:
def misc_read(file_path, df):
    name = file_path.split('\\')[-1].split('.')[0]
    columns = np.nan
    ext = file_path.split('\\')[-1].split('.')[-1]
    size = os.path.getsize(file_path)
    df.loc[0] = [name, columns, ext, size/(1024*1024), file_path]
    return df

In [1]:
def csv_write(file_paths):
    i = 0
    j = 0
    file_no = 0
    while i < len(file_paths):
        actual_err = None
        error = None
        try:
            df = pd.DataFrame({'File_Name':[], 'Columns':[], 'Ext':[], 'Size(in mb)':[], 'File_path':[]})
            if file_paths[i].split('.')[-1] == 'zip':
                df = zip_read(file_paths[i], df)
            elif file_paths[i].split('.')[-1] == 'shp':
                df = shape_read(file_paths[i], df)
            elif file_paths[i].split('.')[-1] in {'xlsx', 'xls'}:
            df = excel_read(file_paths[i], df)
            else:
                df = misc_read(file_paths[i], df)
        except Exception as e:
            error = e
            if type(error) == FileNotFoundError:
                try:
                    temp = open(file_paths[0])
                except Exception as f:
                    actual_err = f
            if (type(error) != FileNotFoundError) or (actual_err == None):
                print(file_paths[i], '\n', f'{error}')
                ex_df = pd.DataFrame({'File_path':[file_paths[i]], 'Exception_type':[type(error)], 'Exception':[error]})
                ex_df.to_csv(r'E:\file_list\Exceptions4.csv', mode='a', index=False, header=False)
            
        if (type(error) != FileNotFoundError) or (actual_err == None):
            df.to_csv(r'E:\file_list\Final_df{}.csv'.format(file_no), mode='a', index=False, header=False)
            i += 1
            j += 1
        if os.path.getsize(r'E:\file_list\Final_df{}.csv'.format(file_no)) >= (10*1024*1024):
            file_no += 1
        if j == 1000:
            time.sleep(3)
            clear_output(wait=True)
            display('Output cleared')
            print(i)
            j = 0